# Desafio (Parte 2) - Extraindo Informações

## Case baseado no desafio do Kaggle: "Instacart Market Basket Analysis"

Esse desafio é baseado em uma competição do Kaggle ([https://www.kaggle.com](https://www.kaggle.com)) de 2017, onde é pedido para fazer a predição de se um produto será comprado por dado usuário no carrinho atual, baseado em suas compras anteriores.

Não está no escopo da aula 



Link para o desafio: [https://www.kaggle.com/c/instacart-market-basket-analysis](https://www.kaggle.com/c/instacart-market-basket-analysis)

## Origem dos Dados:

Os dados mostrados nesse desafio são uma amostra extraída do dataset disponibilizado gratuitamente pela empresa Instacart ([https://www.instacart.com/](https://www.instacart.com/)) **\***. São dados reais do banco de dados da empresa, que foram anonimizados para o uso por pesquisadores e em competições de Data Science.

 
###### * “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from [https://www.instacart.com/datasets/grocery-shopping-2017](https://www.instacart.com/datasets/grocery-shopping-2017) on March 10, 2017.

## Tabelas e Colunas Importantes

As Tabelas e suas respectivas colunas estão mostradas abaixo, junto com a descrição do significado de cada coluna:

* Colunas da Tabela `orders`:

    * ´order_id´: order identifier
    * ´user_id´: customer identifier
    * ´eval_set´: which evaluation set this order belongs in (see SET described below)
    * ´order_number´: the order sequence number for this user (1 = first, n = nth)
    * ´order_dow´: the day of the week the order was placed on
    * ´order_hour_of_day´: the hour of the day the order was placed on
    * ´days_since_prior´: days since the last order, capped at 30 (with NAs for order_number = 1)
    
* Colunas da Tabela `products (50k rows):

    * ´product_id´: product identifier
    * ´product_name´: name of the product
    * ´aisle_id´: foreign key
    * ´department_id´: foreign key

* Colunas da Tabela `aisles (134 rows):

    * ´aisle_id´: aisle identifier
    * ´aisle´: the name of the aisle

* Colunas da Tabela `deptartments (21 rows):

    * ´department_id´: department identifier
    * ´department´: the name of the department

* Colunas da Tabela `order_products__SET (30m+ rows):

    * ´order_id´: foreign key
    * ´product_id´: foreign key
    * ´add_to_cart_order´: order in which each product was added to cart
    * ´reordered´: 1 if this product has been ordered by this user in the past, 0 otherwise

___

# Import

In [1]:
import numpy as np
import os
import pandas as pd

# Objetivo:

Analisar o dataset gerado na Seção 3 e responder às questões.

# Dataset

In [2]:
file_name = os.path.join("data", "dataset.csv")

In [3]:
dataset = pd.read_csv(file_name, sep=",", index_col=0, encoding="utf-8")

C:\Users\allan\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
dataset.head()

,order_id,user_id,order_dow,product_name,aisle,department
0,1187899,1,4,Soda,soft drinks,beverages
1,2757217,67,0,Soda,soft drinks,beverages
2,632715,676,0,Soda,soft drinks,beverages
3,1167274,760,4,Soda,soft drinks,beverages
4,3347074,804,3,Soda,soft drinks,beverages


In [5]:
""" Facilitando a vida """
# baseado em uma thread do Kaggle sobre esse desafio, os dias da semana são definidos na seguinte ordem:
week_days = ["saturday", "sunday", "monday", "tuesday", "wednesday", "thursday", "friday"]

# traduzindo no dataset
dataset.order_dow = dataset.order_dow.map(lambda wd: week_days[wd])

In [6]:
dataset.head()

,order_id,user_id,order_dow,product_name,aisle,department
0,1187899,1,wednesday,Soda,soft drinks,beverages
1,2757217,67,saturday,Soda,soft drinks,beverages
2,632715,676,saturday,Soda,soft drinks,beverages
3,1167274,760,wednesday,Soda,soft drinks,beverages
4,3347074,804,tuesday,Soda,soft drinks,beverages


# Questões 

## A) Quantos usuários únicos fizeram pedidos?

In [7]:
dataset.user_id.unique().shape[0]

117221

## B) Qual o produto que apareceu em mais pedidos?

In [8]:
g = dataset.groupby(["order_id", "product_name"])
product_counts = g.count().reset_index().set_index("order_id").product_name.value_counts()
print("Produto que ocorre em mais pedidos: {} ({} ocorrências)".format(product_counts.argmax(), product_counts.max()))

Produto que ocorre em mais pedidos: Banana (16784 ocorrências)


## C) Quais os usuários que fizeram mais e menos pedidos?

In [9]:
""" Entre com o Código aqui """
g = dataset.groupby(["user_id", "order_id"])
order_counts = g.count().reset_index().set_index("user_id").order_id.value_counts()
print("Usuário com mais pedidos: {} ({} ocorrências)".format(order_counts.argmax(), order_counts.max()))
print("Usuário com menos pedidos: {} ({} ocorrências)".format(order_counts.argmin(), order_counts.min()))

Usuário com mais pedidos: 2887678 (1 ocorrências)
Usuário com menos pedidos: 2887678 (1 ocorrências)


## D) Qual a média e o desvio padrão da quantidade de produtos presentes em um pedido um usuário?

In [10]:
g = dataset.groupby(["order_id"])
product_counts = g.product_name.count()
product_counts.describe()[["mean", "std"]]

mean    10.564191
std      7.945047
Name: product_name, dtype: float64

## E) Qual a média, a mediana, o mínimo e o máximo da quantidade de corredores (`aisles`) por departamento (`department`)?


In [11]:
aisles_per_dept = dataset[["department", "aisle"]].drop_duplicates(["department", "aisle"])
g = aisles_per_dept.groupby("department")
aisle_count = g.aisle.count()
aisle_count.describe()[["mean", "50%", "min", "max"]]

mean     6.380952
50%      5.000000
min      1.000000
max     17.000000
Name: aisle, dtype: float64

## F) Qual o dia da semana em que ocorrem mais pedidos de produtos para cada departmento (`department`)?

In [12]:
g = dataset.groupby(["department", "order_dow"])
product_count_per_dow = g.product_name.count()
product_count_per_dow_table = product_count_per_dow.unstack()
product_count_per_dow_table.apply(lambda x: x.argmax(), axis=1)

department
alcohol            thursday
babies             saturday
bakery             saturday
beverages          saturday
breakfast          saturday
bulk               saturday
canned goods       saturday
dairy eggs         saturday
deli               saturday
dry goods pasta    saturday
frozen             saturday
household          saturday
international      saturday
meat seafood       saturday
missing            saturday
other              saturday
pantry             saturday
personal care      saturday
pets               saturday
produce            saturday
snacks             saturday
dtype: object

##  G) Quais foram os 5% piores e os 5% melhores corredores (`aisle`) em termos de vendas? 

In [13]:
aisles_sales_count = dataset.groupby("aisle").product_name.count()
l_inf, l_sup = aisles_sales_count.describe(percentiles=[.05, .95])[["5%", "95%"]]
l_inf, l_sup

(458.44999999999999, 28386.649999999991)

In [14]:
"""piores corredores"""
aisles_sales_count[aisles_sales_count < l_inf].sort_values()

aisle
beauty                        247
frozen juice                  251
baby accessories              273
baby bath body care           286
kitchen supplies              410
specialty wines champagnes    416
ice cream toppings            450
Name: product_name, dtype: int64

In [15]:
"""melhores corredores"""
aisles_sales_count[aisles_sales_count > l_sup].sort_values(ascending=False)

aisle
fresh fruits                     134576
fresh vegetables                 134559
packaged vegetables fruits        70176
yogurt                            49372
packaged cheese                   37390
water seltzer sparkling water     32647
milk                              29192
Name: product_name, dtype: int64

##  H) Quais foram os 5 produtos mais comprados de cada departamento na terça-feira? 

### Mostre em uma tabela como no exemplo:

| departamento | 1o produto | 2o produto | 3o produto | 4o produto | 5o produto |
|:------------ |:---------- |:---------- |:---------- |:---------- |:---------- |
| depart 01    | prod A     | prod W     | prod L     | prod H     | prod D     |
| depart 02    | prod X     | prod Q     | prod B     | prod S     | prod P     |
| ...          | ...        | ...        | ...        | ...        | ...        |


In [16]:
tuesday_orders = dataset[dataset.order_dow == "tuesday"]
depart_prod_count = pd.crosstab(tuesday_orders.department, tuesday_orders.product_name)

top_5_prods_per_dept = depart_prod_count.apply(lambda x: x.sort_values(ascending=False)[:5].index.tolist(), axis=1)
for dept in top_5_prods_per_dept.index:
    print("Department '{}':".format(dept))
    print(" - " + "\n - ".join(top_5_prods_per_dept.loc[dept]))

Department 'alcohol':
 - Cabernet Sauvignon
 - Sauvignon Blanc
 - India Pale Ale
 - Beer
 - Chardonnay
Department 'babies':
 - Baby Food Stage 2 Blueberry Pear & Purple Carrot
 - Gluten Free SpongeBob Spinach Littles
 - Spinach Peas & Pear Stage 2 Baby Food
 - Peach,  Apricot & Banana Stage 2 Baby Food
 - Broccoli & Apple Stage 2 Baby Food
Department 'bakery':
 - 100% Whole Wheat Bread
 - Organic Bread with 21 Whole Grains
 - Ezekiel 4:9 Bread Organic Sprouted Whole Grain
 - Organic 21 Grain Thin Sliced Bread
 - Original Nooks & Crannies English Muffins
Department 'beverages':
 - Sparkling Water Grapefruit
 - Spring Water
 - Soda
 - Sparkling Natural Mineral Water
 - Lime Sparkling Water
Department 'breakfast':
 - Honey Nut Cheerios
 - Organic Old Fashioned Rolled Oats
 - Raisin Bran Cereal
 - Cereal
 - Cherrios Honey Nut
Department 'bulk':
 - Dried Mango
 - Organic Rolled Oats
 - Organic Black Mission Figs
 - Apricots
 - Vegetable Chips
Department 'canned goods':
 - Organic Black Bean

## I) Quais são os 2 corredores (`aisles`) que são visitados juntos no mesmo pedido mais vezes em uma sexta-feira?

In [17]:
friday_orders = dataset[dataset.order_dow == "friday"]
aisles_visit_count = pd.crosstab(friday_orders.order_id, friday_orders.aisle).applymap(lambda x: 1 if x > 0 else 0)
co_occurrent_aisles = aisles_visit_count.T.dot(aisles_visit_count)
co_occurrent_aisles -= np.triu(co_occurrent_aisles)
co_occurrent_aisles.stack().sort_values(ascending=False).head(1)

aisle             aisle       
fresh vegetables  fresh fruits    5809
dtype: int64

## J) Quais as duplas de produtos que mais saem juntas no pedido (`order_id`) de um usuário em uma sexta-feira?

In [18]:
%%time
friday_orders = dataset[dataset.order_dow == "friday"]
products_count = (pd.crosstab(friday_orders.order_id, friday_orders.product_name) > 0).astype(float)
co_occurrent_products = products_count.T.dot(products_count)
co_occurrent_products -= np.triu(co_occurrent_products)

Wall time: 5min 34s


In [19]:
co_occurrent_products.stack().sort_values(ascending=False).head(1)

product_name          product_name          
Organic Strawberries  Bag of Organic Bananas    409.0
dtype: float64